In [1]:
import pandas as pd
import numpy as np

In [2]:
%%time
prep_path = 'data/TreehouseCompentiumSamples_LibraryPrep.tsv'
prep_df = pd.read_csv(prep_path, sep='\t', index_col=0)

CPU times: user 424 ms, sys: 620 ms, total: 1.04 s
Wall time: 640 ms


In [3]:
%%time
corr_path = 'data/v5_all_by_all.2018-02-04.tsv'
corr_df = pd.read_csv(corr_path, sep='\t', index_col=0)

CPU times: user 2min 26s, sys: 22.4 s, total: 2min 48s
Wall time: 2min 49s


In [4]:
%%time
meta_path = 'data/clinical.tsv'
meta_df = pd.read_csv(meta_path, sep='\t', index_col=0)

CPU times: user 1.04 s, sys: 56 ms, total: 1.09 s
Wall time: 1.05 s


In [6]:
prep_df['libSelType'].value_counts()

polyASelection            11505
riboDepletion               191
presumed riboDepletion       32
unknown                       9
exomeSelection                8
Name: libSelType, dtype: int64